In [ ]:
import pandas as pd
import os

ruta_sisaire = "Datos/SISAIRE"

# ---------------- SISAIRE ---------------- #
def cargar_sisaire(ruta):
    archivos = [f for f in os.listdir(ruta) if f.endswith(('.xlsx', '.csv'))]
    lista_dfs = []
    for archivo in archivos:
        path = os.path.join(ruta, archivo)
        if archivo.endswith('.csv'):
            df = pd.read_csv(path)
        else:
            df = pd.read_excel(path)
        df["fuente"] = "SISAIRE"
        df["archivo_origen"] = archivo
        lista_dfs.append(df)
    df_sisaire = pd.concat(lista_dfs, ignore_index=True)
    print(f"SISAIRE: {len(archivos)} archivos cargados. Forma total: {df_sisaire.shape}")
    print("Columnas detectadas:", list(df_sisaire.columns))
    return df_sisaire

# Cargar ambos datasets

sisaire = cargar_sisaire(ruta_sisaire)


SISAIRE: 24 archivos cargados. Forma total: (2622273, 9)
Columnas detectadas: ['Estacion', 'Fecha inicial', 'Fecha final', 'NO2', 'fuente', 'archivo_origen', 'O3', 'SO2', 'CO']


In [48]:
import pandas as pd
import numpy as np


path = "Datos/IBOCA/IBOCA-PM10-2020-1.xlsx"
columnas_finales = ["Fecha & Hora", "Location", "Concentración", "NowCast", "IBOCA"]

# Leer crudo sin encabezado
df_raw = pd.read_excel(path, header=None)

# 1) Fechas (columna A) desde la fila 8 (índice 7)
df_fechas = df_raw.iloc[7:, [0]].copy()
df_fechas.columns = ["Fecha & Hora"]

# Normalizar a datetime (soporta "1/01/24 0:00", etc.)
df_fechas["Fecha & Hora"] = pd.to_datetime(df_fechas["Fecha & Hora"], errors="coerce", dayfirst=True)

# Función para convertir números con coma decimal y "Sin data" → NaN
def to_num(s):
    if pd.isna(s): 
        return np.nan
    s = str(s).strip()
    if s.lower().startswith("sin data"):
        return np.nan
    # reemplazar coma decimal por punto (p.ej. 24,16 → 24.16)
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

# 2) Recorremos columnas en bloques de 3: (B,C,D)=1..3, (E,F,G)=4..6, ...
bloques = []
n_cols = df_raw.shape[1]

for start_col in range(1, n_cols, 3):
    # Asegurar que el bloque completo existe
    if start_col + 2 >= n_cols:
        break

    # Lugar está en la fila 5 (índice 4) de la 1ª columna del bloque
    lugar = df_raw.iat[4, start_col] if 4 < df_raw.shape[0] else None
    if not isinstance(lugar, str) or lugar.strip() == "" or lugar.strip().lower() == "nan":
        # si no hay lugar, probablemente ya no hay más estaciones útiles
        continue
    lugar = lugar.strip()

    # Datos del bloque desde fila 8 (índice 7)
    vals = df_raw.iloc[7:, start_col:start_col+3].copy()
    vals.columns = ["Concentración", "NowCast", "IBOCA"]

    # Convertir a numéricos (coma decimal, “Sin data”)
    for c in ["Concentración", "NowCast", "IBOCA"]:
        vals[c] = vals[c].apply(to_num)

    # Armar df del bloque con Location repetido y fechas
    df_block = pd.concat([df_fechas.reset_index(drop=True), vals.reset_index(drop=True)], axis=1)
    df_block.insert(1, "Location", lugar)

    # Quitar filas totalmente vacías de medición
    df_block = df_block.dropna(subset=["Concentración", "NowCast", "IBOCA"], how="all")

    # Guardar si tiene contenido
    if not df_block.empty:
        bloques.append(df_block)

   

# 3) Unir todos los bloques
if bloques:
    df_final = pd.concat(bloques, ignore_index=True)
    df_final = df_final[columnas_finales]
else:
    df_final = pd.DataFrame(columns=columnas_finales)

df_final.shape
df_final.head(3)


,Fecha & Hora,Location,Concentración,NowCast,IBOCA
0,2024-01-01 00:00:00,Bolivia,26.0,24.16,76.36
1,2024-01-01 01:00:00,Bolivia,23.0,23.58,75.14
2,2024-01-01 02:00:00,Bolivia,18.0,20.79,69.27


In [42]:
print("Dimensiones del dataset:", df_final.shape)
print("\nTipos de datos:")
print(df_final.dtypes)
print("\nNulos por columna:")
print(df_final.isna().sum())


Dimensiones del dataset: (79019, 5)

Tipos de datos:
Fecha & Hora     datetime64[ns]
Location                 object
Concentración           float64
NowCast                 float64
IBOCA                   float64
dtype: object

Nulos por columna:
Fecha & Hora       57
Location            0
Concentración    1554
NowCast           267
IBOCA             557
dtype: int64


In [43]:
print("\nRango de fechas:")
print(df_final["Fecha & Hora"].min(), " → ", df_final["Fecha & Hora"].max())

# Frecuencia aproximada
df_final["delta_horas"] = df_final["Fecha & Hora"].diff().dt.total_seconds() / 3600
print("\nFrecuencias más comunes (en horas):")
print(df_final["delta_horas"].value_counts().head())



Rango de fechas:
2024-01-01 00:00:00  →  2024-07-01 00:00:00

Frecuencias más comunes (en horas):
delta_horas
1.0    78831
2.0       39
3.0       18
5.0        6
4.0        6
Name: count, dtype: int64


In [44]:
print("\nNúmero de estaciones:", df_final["Location"].nunique())
print("\nTop estaciones por número de registros:")
print(df_final["Location"].value_counts().head(10))



Número de estaciones: 19

Top estaciones por número de registros:
Location
Guaymaral         4372
Móvil 7ma         4371
Tunal             4370
Kennedy           4357
MinAmbiente       4344
Jazmín            4320
Colina            4318
Fontibon          4311
Móvil Fontibón    4311
San Cristobal     4309
Name: count, dtype: int64


In [45]:
print("\n--- Estadísticas de Concentración ---")
print(df_final["Concentración"].describe())

print("\n--- Estadísticas de NowCast ---")
print(df_final["NowCast"].describe())

print("\n--- Estadísticas de IBOCA ---")
print(df_final["IBOCA"].describe())



--- Estadísticas de Concentración ---
count    77465.000000
mean        19.304951
std         13.804879
min          0.000000
25%          8.000000
50%         17.000000
75%         27.000000
max        178.000000
Name: Concentración, dtype: float64

--- Estadísticas de NowCast ---
count    78752.000000
mean        19.993673
std         65.709055
min          0.010000
25%          8.420000
50%         16.990000
75%         27.080000
max       4366.000000
Name: NowCast, dtype: float64

--- Estadísticas de IBOCA ---
count    78462.000000
mean        62.294893
std         71.587111
min          0.010000
25%         34.910000
50%         61.310000
75%         82.480000
max       4349.000000
Name: IBOCA, dtype: float64


In [47]:
print("\nCorrelaciones:")
print(df_final[["Concentración", "NowCast", "IBOCA"]].corr())



Correlaciones:
               Concentración   NowCast     IBOCA
Concentración       1.000000  0.165037  0.405290
NowCast             0.165037  1.000000  0.960455
IBOCA               0.405290  0.960455  1.000000
